In [1]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [2]:
encoder = SentenceTransformer(
    'sentence-transformers_all-MiniLM-L6-v2',
    device='cuda'
)

In [3]:
documents = [
    {"name": "The Time Machine", "description": "A man travels through time and witnesses the evolution of humanity.",
        "author": "H.G. Wells", "year": 1895},
    {"name": "Ender's Game", "description": "A young boy is trained to become a military leader in a war against an alien race.",
        "author": "Orson Scott Card", "year": 1985},
    {"name": "Brave New World", "description": "A dystopian society where people are genetically engineered and conditioned to conform to a strict social hierarchy.",
        "author": "Aldous Huxley", "year": 1932},
    {"name": "The Hitchhiker's Guide to the Galaxy",
        "description": "A comedic science fiction series following the misadventures of an unwitting human and his alien friend.", "author": "Douglas Adams", "year": 1979},
    {"name": "Dune", "description": "A desert planet is the site of political intrigue and power struggles.",
        "author": "Frank Herbert", "year": 1965},
    {"name": "Foundation", "description": "A mathematician develops a science to predict the future of humanity and works to save civilization from collapse.",
        "author": "Isaac Asimov", "year": 1951},
    {"name": "Snow Crash", "description": "A futuristic world where the internet has evolved into a virtual reality metaverse.",
        "author": "Neal Stephenson", "year": 1992},
    {"name": "Neuromancer", "description": "A hacker is hired to pull off a near-impossible hack and gets pulled into a web of intrigue.",
        "author": "William Gibson", "year": 1984},
    {"name": "The War of the Worlds", "description": "A Martian invasion of Earth throws humanity into chaos.",
        "author": "H.G. Wells", "year": 1898},
    {"name": "The Hunger Games", "description": "A dystopian society where teenagers are forced to fight to the death in a televised spectacle.",
        "author": "Suzanne Collins", "year": 2008},
    {"name": "The Andromeda Strain", "description": "A deadly virus from outer space threatens to wipe out humanity.",
        "author": "Michael Crichton", "year": 1969},
    {"name": "The Left Hand of Darkness", "description": "A human ambassador is sent to a planet where the inhabitants are genderless and can change gender at will.",
        "author": "Ursula K. Le Guin", "year": 1969},
    {"name": "The Three-Body Problem", "description": "Humans encounter an alien civilization that lives in a dying system.",
        "author": "Liu Cixin", "year": 2008}
]

In [4]:
qdrant = QdrantClient(
    host='localhost',
    port=6333
)

In [6]:
qdrant.recreate_collection(
	collection_name='my_books',
	vectors_config=models.VectorParams(
		size=encoder.get_sentence_embedding_dimension(),
		distance=models.Distance.COSINE
	)
)

True

In [7]:
records = []

for idx, doc in enumerate(documents):
    record = models.Record(
        id=idx,
        vector=encoder.encode(doc['description']).tolist(),
        payload=doc
    )
    records.append(record)

In [8]:
records

[Record(id=0, payload={'name': 'The Time Machine', 'description': 'A man travels through time and witnesses the evolution of humanity.', 'author': 'H.G. Wells', 'year': 1895}, vector=[-0.03813490271568298, 0.04517786577343941, -0.022021334618330002, 0.05923553928732872, 0.006653958465903997, 0.018390005454421043, 0.06810281425714493, -0.041662316769361496, 0.044377703219652176, -0.025251789018511772, -0.009794259443879128, -0.022916490212082863, -0.11903539299964905, 0.031319014728069305, -0.057532891631126404, -0.008934100158512592, -0.10448412597179413, 0.007782082073390484, 0.05709609016776085, -0.05635252967476845, -0.005898033734411001, 0.026749474927783012, -0.04767239838838577, -1.9592928310885327e-06, -0.03601926937699318, 0.001294949441216886, 0.06238840892910957, 0.0017686357023194432, 0.05468185618519783, 0.021232400089502335, 0.07367482036352158, -0.012031602673232555, -0.032027531415224075, -0.03724589943885803, -0.045938167721033096, 0.007532390300184488, 0.00135790638159

In [9]:
qdrant.upload_records(
    collection_name='my_books',
    records=records
)

In [11]:
hits = qdrant.search(
	collection_name='my_books',
	query_vector=encoder.encode('alien invasion').tolist(),
	limit=3
)
for hit in hits:
	print(hit.payload, 'score:', hit.score)

{'author': 'H.G. Wells', 'description': 'A Martian invasion of Earth throws humanity into chaos.', 'name': 'The War of the Worlds', 'year': 1898} score: 0.5700933
{'author': 'Douglas Adams', 'description': 'A comedic science fiction series following the misadventures of an unwitting human and his alien friend.', 'name': "The Hitchhiker's Guide to the Galaxy", 'year': 1979} score: 0.5040468
{'author': 'Liu Cixin', 'description': 'Humans encounter an alien civilization that lives in a dying system.', 'name': 'The Three-Body Problem', 'year': 2008} score: 0.4590294


In [12]:
hits = qdrant.search(
	collection_name='my_books',
	query_vector=encoder.encode('alien invasion').tolist(),
	query_filter=models.Filter(
		must=[
			models.FieldCondition(
				key='year',
				range=models.Range(
					gte=2000
				)
			)
		]
	),
	limit=1
)
for hit in hits:
	print(hit.payload, 'score:', hit.score)

{'author': 'Liu Cixin', 'description': 'Humans encounter an alien civilization that lives in a dying system.', 'name': 'The Three-Body Problem', 'year': 2008} score: 0.4590294
